<a href="https://colab.research.google.com/github/PauloTelles046/IA-Geradora-Diagnosticos/blob/main/Projeto_IA_Geradora_de_Diagn%C3%B3sticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Autor:** Paulo Henrique Telles Ferreira

#**Instalação das bibliotecas**



In [124]:
#!pip install -q -U google-generativeai

# **Importação das bibliotecas**

In [125]:
import json #biblioteca utilizada para manipulação JSON
import re #biblioteca utilizada para validação
import google.generativeai as genai
from google.colab import userdata
import textwrap
from IPython.display import display
from IPython.display import Markdown

# **Configuração do modelo**

In [126]:

api_key=userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

# Set up the model
generation_config = {
  "temperature": 0.8,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)


# **Função para geração de diagnósticos**
Nesta função, são informados os dados do paciente como sintomas, idade, sexo, altura e peso.
Estes dados são enviados para o Gemini em forma de prompt e então o resultado é retornado em formato JSON.


In [127]:
#Função para gerar diagnósticos
def gerar_diagnostico(sintoma,idade,sexo,altura,peso):
    #Se o sexo for informado
    if sexo!="":
      entrada=f"input: Tenho {idade} anos, sou do sexo {sexo}, tenho {altura} de altura e peso {peso}. {sintoma}"
    #Se o sexo não for informado
    else:
      entrada=f"input: Tenho {idade} anos, tenho {altura} de altura e peso {peso}. {sintoma}"

    prompt_parts = [
  "Classifique sempre se a consulta foi realizada com sucesso ou não no campo \"response\". Responda com 1 para sim e 0 para não.",
  "input: Tenho 55 anos, sou do sexo masculino, tenho 1,70m de altura e tenho 80kg sentido uma dor fixa no peito, acho que tenho sentido uma azia, tenho transpirado bastante, tenho tido falta de ar, tenho tido náuseas e ontem desmaiei.",
  "Resposta [    {        \"diagnostic\":\"Este pode ser o sintoma de um possível infarto. Recomendo procurar um cardiologista\",        \"exams\":[\"Ecocardiografia\", \"MAPA\", \"Holter\"],        \"symptoms\":[\"azia\",\"transpiração excessiva\",\"falta de ar\",\"náusea\",\"desmaios\"],        \"response\":1    }]",
  "Clínicas Essas clínicas podem ajudar:\nClínica cardiologista: https://www.googleadservices.com/pagead/aclk?sa=L&ai=DChcSEwjSkcncjIGGAxUjjsIIHWIrASUYABAAGgJqZg&ase=2&gclid=Cj0KCQjw6PGxBhCVARIsAIumnWaPLpA3mE27KhoAxAJmoWS8rl_dnx-URmzS4et5HRbVU8vesBi6XOwaAjMOEALw_wcB&ohost=www.google.com&cid=CAESVeD2tQqR3e7xmOcK_TRe1vbztXMdh0E5g2x8mu55l4ocZeTnG4ZlHWR5ICiNYCl9JFIpZu_uxqJ9PfFKCKMwNtqfeZfu6uBRCW3ra5_5-avRJP1CFN8&sig=AOD64_0UTEbzGGLoJr5M3euswFKcGMWxEA&q&nis=4&adurl&ved=2ahUKEwjgt77cjIGGAxUsrJUCHRzEDsUQ0Qx6BAgIEAE\n\nHospital Cardiológico Constantine: https://hospitalcostantini.com.br/\n\nDr Carlos Eduardo Passarini: https://carloseduardopassarini.com.br/?gad_source=1&gclid=Cj0KCQjw6PGxBhCVARIsAIumnWacHy_KWzwRPXhhzJH6RlpBgSX0PnGNpcgulDjDVPQDqiIHVIb9Yi4aAgAcEALw_wcB\n\nCateterismo Cardíaco em Nova Iguaçu. Agende uma Consulta | doctoralia.com.br: https://www.doctoralia.com.br/servicos-de-tratamento/cateterismo-cardiaco/nova-iguacu\n\nHospital São Mateus: https://hospitalsaomatheus.com.br/",
  "input: Minha filha tem 5 anos, é do sexo feminino, tem 110,00 cm\\n de altura e tem 37,74 kg. Tem sentido febre, dor de garganta, tosse, dor no corpo e muita dor de cabeça",
  "Resposta [    {        \"diagnostic\":\"Estes são os sintomas de uma gripe. Recomendo procurar um pediatra, dependendo da idade do seu filho, ou um infectologista.\",        \"exams\":[\"radiografia de tórax\", \"exame de sangue\", \"pesquisa de anticorpos anti-influenza A e B\", \"PCR\", \"Pesquisa rápida de antígenos virais.\"],        \"symptoms\":[\"febre\",\"dor de garganta\",\"tosse\",\"dor no corpo\",\"muita dor de cabeça\"],        \"response\":1    }]",
  "Clínicas Essas clínicas podem ajudar:\nClínica Médica M.M Saúde: https://www.clinicammsaude.com.br/?utm_source=google&utm_medium=gmn&utm_campaign=&utm_term=botao_website\n\nClínica Médica Menino Jesus: https://www.instagram.com/clinicameninojesusrj/",
  "input: Tenho 30 anos, tenho 1,45m de altura, sou do sexo feminino e peso 70kg. Eu tenho sentido dor no ouvido e muita dor de cabeça",
  "Resposta [    {        \"diagnostic\":\"Esta pode ser uma inflamação no ouvido ou uma infecção no ouvido. Eu recomendo que você consulte um clínico geral ou otorrinolaringologista\",        \"exams\":[\"Otoscopia\"],        \"symptoms\":[\"dor de ouvido\", \"dor de cabeça\"],        \"response\":1    }]",
  "Clínicas Essas clínicas podem ajudar:\nOtorrinolaringologia São Paulo: https://www.otorrinosp.com.br/\n\nOtorrino DF: http://www.otorrinodf.com.br/\n\nOtorrinolaringologista - Dr. Marcus Túlio C. Lessa CRM: https://www.marcustuliolessa.com.br/",
  f"input:{entrada} ",
  "Resposta ",
]

    response = model.generate_content(prompt_parts)
    return response.text

# **Função para conversão das clínicas em formato JSON**
Esta função foi criada para resolver o problema da lista de clínicas. Estas foram melhor geradas pelo modelo em formato discursivo. Esta função transforma a lista de clínicas sugeridas pelo modelo como resposta ao prompt de geração de diagnóstico em uma lista JSON.

In [128]:
#Função para converter as clínicas sugeridas em formato JSON
def converter_clinicas_json(clinicas):
    prompt_parts = [
    "Separe cada clínica a seguir no formato {\"name\":\"nome da clínica\",\"link\":\"link da clínica\"}",
    "input: Essas clínicas podem ajudar:\nClínica cardiologista: https://www.googleadservices.com/pagead/aclk?sa=L&ai=DChcSEwjSkcncjIGGAxUjjsIIHWIrASUYABAAGgJqZg&ase=2&gclid=Cj0KCQjw6PGxBhCVARIsAIumnWaPLpA3mE27KhoAxAJmoWS8rl_dnx-URmzS4et5HRbVU8vesBi6XOwaAjMOEALw_wcB&ohost=www.google.com&cid=CAESVeD2tQqR3e7xmOcK_TRe1vbztXMdh0E5g2x8mu55l4ocZeTnG4ZlHWR5ICiNYCl9JFIpZu_uxqJ9PfFKCKMwNtqfeZfu6uBRCW3ra5_5-avRJP1CFN8&sig=AOD64_0UTEbzGGLoJr5M3euswFKcGMWxEA&q&nis=4&adurl&ved=2ahUKEwjgt77cjIGGAxUsrJUCHRzEDsUQ0Qx6BAgIEAE\n\nHospital Cardiológico Constantine: https://hospitalcostantini.com.br/\n\nDr Carlos Eduardo Passarini: https://carloseduardopassarini.com.br/?gad_source=1&gclid=Cj0KCQjw6PGxBhCVARIsAIumnWacHy_KWzwRPXhhzJH6RlpBgSX0PnGNpcgulDjDVPQDqiIHVIb9Yi4aAgAcEALw_wcB\n\nCateterismo Cardíaco em Nova Iguaçu. Agende uma Consulta | doctoralia.com.br: https://www.doctoralia.com.br/servicos-de-tratamento/cateterismo-cardiaco/nova-iguacu\n\nHospital São Mateus: https://hospitalsaomatheus.com.br/",
    "output: \"clinics\":[\n            {\n                \"name\":\"Clínica cardiologista\",\n                \"link\":\" https://www.googleadservices.com/pagead/aclk?sa=L&ai=DChcSEwjSkcncjIGGAxUjjsIIHWIrASUYABAAGgJqZg&ase=2&gclid=Cj0KCQjw6PGxBhCVARIsAIumnWaPLpA3mE27KhoAxAJmoWS8rl_dnx-URmzS4et5HRbVU8vesBi6XOwaAjMOEALw_wcB&ohost=www.google.com&cid=CAESVeD2tQqR3e7xmOcK_TRe1vbztXMdh0E5g2x8mu55l4ocZeTnG4ZlHWR5ICiNYCl9JFIpZu_uxqJ9PfFKCKMwNtqfeZfu6uBRCW3ra5_5-avRJP1CFN8&sig=AOD64_0UTEbzGGLoJr5M3euswFKcGMWxEA&q&nis=4&adurl&ved=2ahUKEwjgt77cjIGGAxUsrJUCHRzEDsUQ0Qx6BAgIEAE\"\n            },\n            {\n                \"name\":\"Hospital Cardiológico Constantine\",\n                \"link\":\"https://hospitalcostantini.com.br/\"\n            },\n            {\n                \"name\":\"Dr Carlos Eduardo Passarini\",\n                \"link\":\" https://carloseduardopassarini.com.br/?gad_source=1&gclid=Cj0KCQjw6PGxBhCVARIsAIumnWacHy_KWzwRPXhhzJH6RlpBgSX0PnGNpcgulDjDVPQDqiIHVIb9Yi4aAgAcEALw_wcB\"\n            },\n            {\n                \"name\":\"Cateterismo Cardíaco em Nova Iguaçu. Agende uma Consulta | doctoralia.com.br\",\n                \"link\":\"https://www.doctoralia.com.br/servicos-de-tratamento/cateterismo-cardiaco/nova-iguacu\"\n            },\n            {\n                \"name\":\"Hospital São Mateus\",\n                \"link\":\"https://hospitalsaomatheus.com.br/\"\n            }\n        ]",
    "input: Essas clínicas podem ajudar:\nClínica Médica M.M Saúde: https://www.clinicammsaude.com.br/?utm_source=google&utm_medium=gmn&utm_campaign=&utm_term=botao_website\n\nClínica Médica Menino Jesus: https://www.instagram.com/clinicameninojesusrj/",
    "output: \"clinics\":[\n            {\n                \"name\":\"Clínica Médica M.M Saúde\",\n                \"link\":\"https://www.clinicammsaude.com.br/?utm_source=google&utm_medium=gmn&utm_campaign=&utm_term=botao_website\"\n            },\n            {\n                \"name\":\"Clínica Médica Menino Jesus\",\n                \"link\":\"https://www.instagram.com/clinicameninojesusrj/\"\n            }\n        ]",
    f"input:{clinicas}",
    "output: ",
    ]
    response = model.generate_content(prompt_parts)
    txt_resp=response.text

    #Limpeza do resultado recebido
    txt_resp=txt_resp.replace('```json','').replace('```','').replace('"clinics":','')

    #Conversão do resultado recebido para formato JSON
    clinicas_json=json.loads(txt_resp)
    return clinicas_json


# **Função para tradução das respostas do usuário**
Esta função foi desenvolvida para que o usuário possa informar ao sistema o seu pedido da sua forma. É enviado um prompt para o Gemini para que classifique o pedido do usuário entre SINTOMA, REALIZAR_DIAGNOSTICO, ENCERRAR ou INVÁLIDO. Desta maneira, o sistema pode executar as funções necessárias dependendo do tipo de pedido do usuário.

In [137]:
#Recebe a requisicao e retorna a constante respectiva
#Isso faz com que o programa se torne mais conversacional
def traduzir_requisicao(requisicao):
    prompt_parts = [
  "Classifique as seguintes requisições como SINTOMA, REALIZAR_DIAGOSTICO ou INVÁLIDO se não puder ser classificado como nenhuma das anteriores.",
  "input: Estou sentindo tosse e dor de cabeça",
  "output: SINTOMA",
  "input: Meu ouvido está doendo",
  "output: SINTOMA",
  "input: Eu ainda quero te falar alguns sintomas que estou sentindo",
  "output: REALIZAR_DIAGOSTICO",
  "input: Como posso te falar como estou me sentindo?",
  "output: REALIZAR_DIAGOSTICO",
  "input: ",
  "output: INVÁLIDO",
  "input: Quero me diagnosticar",
  "output: REALIZAR_DIAGOSTICO",
  f"input:{requisicao}",
  "output: ",
]


    response = model.generate_content(prompt_parts)

    #Retorna o resultado sem espaços
    return response.text.replace(" ","").replace("\n","")


# **Função para validação de números inteiros**
Função criada para validar strings que apenas devem possuir números inteiros. A função retorna True para valores que estão de acordo com o padrão e False para valores fora do padrão.

In [130]:
#Faz a validação de números inteiros
def valida_inteiros(valor):
    formato=r"[0-9]"
    result=re.search(formato,valor)
    if result!=None:
        return True
    else:
        return False

# **Função para validação de números decimais**
Função criada para validar strings que apenas devem possuir números decimais. A função retorna True para valores que estão de acordo com o padrão e False para valores fora do padrão.

In [131]:
#Faz a validação de números decimais
def valida_decimais(valor):
    formato1=r"[0-9][,.]" #Formato com números decimais
    formato2=r"[0-9]" #Formato com números inteiros

    #Foi utilizado um if para validar os dois formatos. Com isso, foi possível digitar tanto números decimais quanto números inteiros
    result=re.search(formato1, valor)
    if result!=None:
      return True
    else:
      if re.search(formato2,valor) !=None:
        return True
      return False

# **Função para cadastro de paciente**
Foi feito o cadastro dos dados do paciente para que o modelo possa ter uma acurácia melhor. Com dados precisos como idade, sexo, altura e peso, o modelo poderá fazer uma análise melhor do diagnóstico.

In [132]:
def executar_cadastramento():
    #Inicialização de variáveis
    infos={}
    nome=""
    genero=""
    idade=""
    altura=""
    peso=""

    #Inserção das informações pessoais
    display(to_markdown("Para começarmos, me diga seu nome:"))
    nome=input("\tEu:")
    display(to_markdown("<br>Agora farei algumas perguntas para que possamos iniciar sua consulta:<br>"))

    #Verificação de gênero. Foram implementadas funcionalidades para permitir que o usuário
    #informe somente caso se sinta confortável
    display(to_markdown(f"{nome}, poderia me informar seu gênero? (S/N)"))
    resp=input("\tEu:")
    if resp.lower()=="s":
        genero=""
        while True:
            #Aceita apenas os valores m ou f
            display(to_markdown("Gênero:(M/F)"))
            genero=input("\tEu:")
            genero=genero.upper() #A informação recebida é convertida para maiúsculo
                                  #para que possam ser digitadas tanto letras maiúsculas quanto minúsculas

            #Caso seja detectado um valor válido, este é convertido para um formato de texto que
            #será melhor interpretado no prompt
            if genero=="M":
                genero="masculino"
                break
            elif genero=="F":
                genero="feminino"
                break

    elif resp.lower()=="n":
        pass
    else:
        pass

    #Foi utilizada a estrutura while para que um loop seja criado até que o usuário informe um valor válido

    #While para somente aceitar valores válidos para idade
    while not valida_inteiros(idade):
        display(to_markdown(f"{nome}, qual a sua idade?\n"))
        idade=input("\tEu:")

    #While para somente aceitar valores válidos para altura
    while not valida_decimais(altura):
        display(to_markdown(f"Qual sua altura? Ex(1.70)\n"))
        altura=input("\tEu:")

    #While para somente aceitar valores válidos para peso
    while not valida_decimais(peso):
        display(to_markdown(f"Qual seu peso? Ex(80)\n"))
        peso=input("\tEu:")

    #Adiciona a unidade de medida ao peso
    if not "kg" in peso:
        peso+="kg"

    #Adiciona as informações ao dicionário
    infos["nome"]=nome
    infos["genero"]=genero
    infos["idade"]=idade
    infos["altura"]=altura
    infos["peso"]=peso

    return infos


# **Função para melhoria do texto**
Esta função faz com que o texto possa ser exibido em formato html. Desta forma, ele se torna mais legível.

In [133]:
def to_markdown(text):
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# **Função para realizar e exibir o diagnóstico**
Esta função realiza o diagnóstico com base nas informações dadas pelo paciente e realiza o tratamento da resposta para criar uma exibição mais agradável.

In [134]:
def executar_diagnostico(sintomas,idade,genero,altura,peso):
    #Diagnóstico é gerado
    display(to_markdown("<p>Gerando diagnóstico...</p>"))
    resultado= gerar_diagnostico(sintomas,idade,genero,altura,peso)

    #Encontra as clínicas na string vinda como resultado
    indice=resultado.find("Clínicas")
    clinicas=resultado[indice:]
    clinicas=converter_clinicas_json(clinicas)

    #Remove as clínicas do diagnóstico
    resultado=resultado.replace(resultado[indice:],'')

    #Converte o resultado da geração de diagnóstico em JSON
    resultado_json=json.loads(resultado)[0]

    #Exibe o diagnóstico
    diagnostico=resultado_json["diagnostic"]
    exames=resultado_json["exams"]

    display(to_markdown(f"<br><strong>Diagnóstico:</strong> {diagnostico}"))

    #If para que a grafia possa ser melhorada. Se for mais de um item, será referido como exames.
    if len(exames)>1:
        display(to_markdown(f"<br>Os seguintes exames podem ser feitos:"))
    else:
        #Se for um item, se referirá como exame
        display(to_markdown(f"<br><strong>Recomendo fazer este exame:</strong>"))

    #Conversão dos exames em itens de uma lista HTML
    exams_items=""
    for item in exames:
        exams_items+=f"<li>{item}</li>"

    #Os exames convertidos em itens são adicionados em uma lista HTML e exibidos
    exams_html=f"<ul>{exams_items}</ul>"
    display(to_markdown(exams_html))

    display(to_markdown("<br>As clínicas que podem ser consultadas são:<br>"))

    #Faz a iteração pela lista de clínicas e as exibe em um formato legível
    for clinica in clinicas:
        name=clinica["name"]
        link=clinica["link"]
        display(to_markdown(f"<h3>{name}</h3> Site da clínica: <a>{link}</a><br>"))


# **Execução do programa**
O programa começa a ser executado. Os dados são pedidos ao usuário e posteriormente é iniciado o atendimento, onde o usuário pode informar seus sintomas e receber seu diagnóstico.

In [142]:

#Inicialização das variáveis
trad_pedido=""
infos=None
nome=""
genero=""
idade=""
altura=""
peso=""

#Apresentação
display(to_markdown("Olá! Sou sua IA Médica Assistente."))
display(to_markdown("Diga o que você está sentindo que irei ajudar! :)"))

infos= executar_cadastramento() #Executa a função de cadastramento

#Adiciona as informações recebidas em variáveis
nome=infos["nome"]
genero=infos["genero"]
idade=infos["idade"]
altura=infos["altura"]
peso=infos["peso"]


#Finalização do cadastramento e início do atendimento
display(to_markdown(f"Pronto! Foi tudo.Em que posso te ajudar, {nome}?<br>"))
pedido=input("\tEu:")
display(to_markdown("<p>Processando...</p>"))
trad_pedido=traduzir_requisicao(pedido)

#Tratamento das opções

#Executa diagnóstico se for informado um sintoma
if "SINTOMA" in trad_pedido:
    executar_diagnostico(pedido,idade,genero,altura,peso)

#Começa pedindo que informe os sintomas e depois executa o diagnóstico
elif "REALIZAR_DIAGOSTICO" in trad_pedido:
    display(to_markdown(f"Por favor, me diga o que está sentindo {nome}:<br>"))
    sintomas=input("\tEu:")
    executar_diagnostico(sintomas,idade,genero,altura,peso)

#Caso a resposta seja inválida
elif "INVÁLIDO" in trad_pedido:
    display(to_markdown("Não entendi seu pedido."))
    display(to_markdown("Encerrando programa..."))

else:
    pass

display(to_markdown("Muito obrigado por me escolher para realizar seu diagnóstico. Até mais!"))

> Olá! Sou sua IA Médica Assistente.

> Diga o que você está sentindo que irei ajudar! :)

> Para começarmos, me diga seu nome:

	Eu:Carlota


> <br>Agora farei algumas perguntas para que possamos iniciar sua consulta:<br>

...

> <ul><li>exame físico</li><li>tomografia computadorizada</li><li>ressonância magnética</li></ul>

> <br>As clínicas que podem ser consultadas são:<br>

> <h3>Neurologia Especializada</h3> Site da clínica: <a>https://neurologiaespecializada.com.br/?utm_source=google&utm_medium=gmn&utm_campaign=&utm_term=botao_website</a><br>

> <h3>Clínica de Neurologia do Dr. Leandro Teles</h3> Site da clínica: <a>https://clinicaneurologiadrleandro.com.br/</a><br>

> Muito obrigado por me escolher para realizar seu diagnóstico. Até mais!